# Lars Ulrich Transformer Composer (ver. 5.5)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/Lars-Ulrich-Transformer
!pip install huggingface_hub
!pip install torch
!pip install einops
!pip install torch-summary
!pip install tqdm
!pip install matplotlib
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Lars Ulrich Transformer modules...')

import os
import copy
import pickle
import random
import secrets
import statistics
from time import time
import tqdm

print('=' * 70)
print('Loading main Lars Ulrich Transformer modules...')
import torch

%cd /content/Lars-Ulrich-Transformer

import TMIDIX

from x_transformer import *

%cd /content/
print('=' * 70)
print('Loading aux Lars Ulrich Transformer modules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

from midi2audio import FluidSynth
from IPython.display import Audio, display

from huggingface_hub import hf_hub_download

from google.colab import files

print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

In [ ]:
#@title Load Lars Ulrich Transformer Small Model

#@markdown Fast model, 32 layers, 201k MIDIs training corpus

full_path_to_model_checkpoint = "/content/Lars-Ulrich-Transformer/Model/Lars_Ulrich_Transformer_Small_Trained_Model_51000_steps_0.5062_loss_0.8283_acc.pth" #@param {type:"string"}

#@markdown Model precision option

model_precision = "bfloat16" # @param ["bfloat16", "float16", "float32"]

#@markdown bfloat16 == Third precision/triple speed (if supported, otherwise the model will default to float16)

#@markdown float16 == Half precision/double speed

#@markdown float32 == Full precision/normal speed

display_tokens_embeddings = True #@param {type:"boolean"}

print('=' * 70)
print('Loading Lars Ulrich Transformer Small Pre-Trained Model...')
print('Please wait...')
print('=' * 70)

if os.path.isfile(full_path_to_model_checkpoint):
  print('Model already exists...')

else:
  hf_hub_download(repo_id='asigalov61/Lars-Ulrich-Transformer',
                  filename='Lars_Ulrich_Transformer_Small_Trained_Model_51000_steps_0.5062_loss_0.8283_acc.pth',
                  local_dir='/content/Lars-Ulrich-Transformer/Model/',
                  local_dir_use_symlinks=False)

print('=' * 70)
print('Instantiating model...')

torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda'

if model_precision == 'bfloat16' and torch.cuda.is_bf16_supported():
  dtype = 'bfloat16'
else:
  dtype = 'float16'

if model_precision == 'float16':
  dtype = 'float16'

if model_precision == 'float32':
  dtype = 'float32'

ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

SEQ_LEN = 2048

# instantiate the model

model = TransformerWrapper(
    num_tokens = 659,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 1024, depth = 32, heads = 8, attn_flash=True)
)

model = AutoregressiveWrapper(model)

model = torch.nn.DataParallel(model)

model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(full_path_to_model_checkpoint))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

print('Model will use', dtype, 'precision...')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

if display_tokens_embeddings:
  # Plot Token Embeddings
  tok_emb = model.module.net.token_emb.emb.weight.detach().cpu().tolist()

  cos_sim = metrics.pairwise_distances(
    tok_emb, metric='cosine'
  )
  plt.figure(figsize=(7, 7))
  plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
  im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
  plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
  plt.xlabel("Position")
  plt.ylabel("Position")
  plt.tight_layout()
  plt.plot()
  plt.savefig("/content/Lars-Ulrich-Transformer-Small-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (LOAD SEED MIDI)

In [ ]:
#@title Load Seed MIDI

#@markdown Press play button to to upload your own seed MIDI or to load one of the provided sample seed MIDIs from the dropdown list below

select_seed_MIDI = "Upload your own custom MIDI" #@param ["Upload your own custom MIDI", "Nothing Else Matters", "Honesty", "House Of The Rising Sun", "Sharing The Night Together"]
number_of_prime_tokens = 300 # @param {type:"slider", min:30, max:5000, step:5}

#@markdown Seed MIDI controls

MIDI_timings_multiplier = 1 #@param {type:"slider", min:0.1, max:2, step:0.1}
MIDI_pitches_transpose_value = 0 #@param {type:"slider", min:-6, max:5, step:1}
MIDI_velocities_multiplier = 1 #@param {type:"slider", min:0.1, max:2, step:0.1}

#@markdown Other settings

render_MIDI_to_audio = False #@param {type:"boolean"}

print('=' * 70)
print('Lars Ulrich Transformer Seed MIDI Loader')
print('=' * 70)

f = ''

if select_seed_MIDI != "Upload your own custom MIDI":
  print('Loading seed MIDI...')
  f = '/content/Lars-Ulrich-Transformer/Seeds/'+select_seed_MIDI+'.mid'
  score = TMIDIX.midi2single_track_ms_score(open(f, 'rb').read(), recalculate_channels=False)

else:
  print('Upload your own custom MIDI...')
  print('=' * 70)
  uploaded_MIDI = files.upload()
  if list(uploaded_MIDI.keys()):
    score = TMIDIX.midi2single_track_ms_score(list(uploaded_MIDI.values())[0], recalculate_channels=False)
    f = list(uploaded_MIDI.keys())[0]

if f != '':

  print('=' * 70)
  print('File:', f)
  print('=' * 70)

  #=======================================================
  # START PROCESSING

  melody_chords_f = []
  melody_chords_f1 = []
  chords = []

  # INSTRUMENTS CONVERSION CYCLE
  events_matrix = []
  itrack = 1
  patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

  patch_map = [
              [0, 1, 2, 3, 4, 5, 6, 7], # Piano
              [24, 25, 26, 27, 28, 29, 30], # Guitar
              [32, 33, 34, 35, 36, 37, 38, 39], # Bass
              [40, 41], # Violin
              [42, 43], # Cello
              [16, 17, 18, 19, 20], # Organ
              [56, 57, 58, 59, 60], # Trumpet
              [64, 65, 66, 67, 68, 69, 70, 71], # Sax
              [72, 73, 74, 75, 76, 77, 78], # Flute
              [-1], # Drums
              ]

  while itrack < len(score):
      for event in score[itrack]:
          if event[0] == 'note' or event[0] == 'patch_change':
              events_matrix.append(event)
      itrack += 1

  events_matrix.sort(key=lambda x: x[1])

  events_matrix1 = []

  for event in events_matrix:
          if event[0] == 'patch_change':
              patches[event[2]] = event[3]

          if event[0] == 'note':
              event.extend([patches[event[3]]])
              once = False

              for p in patch_map:
                  if event[6] in p and event[3] != 9: # Except the drums
                      event[3] = patch_map.index(p)
                      once = True

              if not once and event[3] != 9: # Except the drums
                  event[3] = 0 # All other instruments/patches channel
                  event[5] = max(80, event[5])

              if event[3] < 10: # We won't write chans 12-16 for now...
                  events_matrix1.append(event)

  if len(events_matrix1) > 0:

      #=======================================================
      # PRE-PROCESSING

      if len(events_matrix1) > 0:

        # recalculating timings
        for e in events_matrix1:
            e[1] = int(e[1] / 8) # Max 2 seconds for start-times
            e[2] = int(e[2] / 32) # Max 4 seconds for durations

        # Sorting by pitch, then by start-time
        events_matrix1.sort(key=lambda x: x[4], reverse = True)
        events_matrix1.sort(key=lambda x: x[3])
        events_matrix1.sort(key=lambda x: x[1])

        #=======================================================
        # FINAL PRE-PROCESSING

        melody_chords = []
        melody_chords_d = []

        pe = events_matrix1[0]
        ped = events_matrix1[0]

        for e in events_matrix1:

            if e[3] != 9:

              # Cliping all values...
              time = max(0, min(255, int((e[1]-pe[1]) * MIDI_timings_multiplier)))
              dur = max(1, min(127, int(e[2] * MIDI_timings_multiplier)))
              cha = max(0, min(9, e[3]))
              ptc = max(1, min(127, e[4] + MIDI_pitches_transpose_value))

              # Shifting drums pitches
              if cha != 9:
                  aug_ptc = ptc
              else:
                  aug_ptc = ptc + 128

              # Calculating octo-velocity
              vel = max(8, min(127, int(e[5] * MIDI_velocities_multiplier)))
              velocity = round(vel / 15)-1

              # Writing final note
              melody_chords.append([time, dur, cha, aug_ptc, velocity])

              pe = e

            # Cliping all values...
            time = max(0, min(255, int((e[1]-ped[1]) * MIDI_timings_multiplier)))
            dur = max(1, min(127, int(e[2] * MIDI_timings_multiplier)))
            cha = max(0, min(9, e[3]))
            ptc = max(1, min(127, e[4] + MIDI_pitches_transpose_value))

            # Shifting drums pitches
            if cha != 9:
                aug_ptc = ptc
            else:
                aug_ptc = ptc + 128

            # Calculating octo-velocity
            vel = max(8, min(127, int(e[5] * MIDI_velocities_multiplier)))
            velocity = round(vel / 15)-1

            # Writing final note
            melody_chords_d.append([time, dur, cha, aug_ptc, velocity])

            ped = e

        #=======================================================
        # FINAL PROCESSING
        #=======================================================

        # Break between compositions / Intro seq

        melody_chords_f.extend([658, 658, 658, 658, 658])

        #=======================================================

        # TOTAL DICTIONARY SIZE 658+1=659

        #=======================================================
        # MAIN PROCESSING CYCLE
        #=======================================================

        for m in melody_chords_d:

            # WRITING EACH NOTE HERE
            time = m[0]
            dur = m[1]
            cha = m[2]
            ptc = m[3]
            vel = m[4]

            melody_chords_f.extend([cha, time+10, dur+266, ptc+394, vel+650])
            melody_chords_f1.append([cha, time+10, dur+266, ptc+394, vel+650])

        cho = []

        for m in melody_chords:

            # WRITING EACH CHORD HERE
            time = m[0]
            dur = m[1]
            cha = m[2]
            ptc = m[3]
            vel = m[4]

            if time == 0:
              cho.append([cha, time+10, dur+266, ptc+394, vel+650])
            else:
              if len(cho) > 0:
                chords.append(cho)
              cho = []
              cho.append([cha, time+10, dur+266, ptc+394, vel+650])

        if len(cho) > 0:
            chords.append(cho)
  #=======================================================

  melody_chords_f1 = melody_chords_f1[:(number_of_prime_tokens // 5)]
  melody_chords_f = melody_chords_f[:number_of_prime_tokens]

  #=======================================================

  print('Composition stats:')
  print('Composition has', len(melody_chords_f1), 'notes')
  print('Composition has', len([y for y in melody_chords_d if y[0] != 0]), 'chords')
  print('Composition has', len(melody_chords_f), 'tokens')
  print('=' * 70)

  #=======================================================

  song = melody_chords_f

  song_f = []

  time = 0
  dur = 0
  vel = 90
  pitch = 0
  channel = 0

  for ss in song:

    if ss >= 0 and ss < 10:

        channel = ss

    if ss >= 10 and ss < 266:

        time += (ss-10) * 8

    if ss >= 266 and ss < 394:

        dur = (ss-266) * 32

    if ss >= 394 and ss < 650:

        pitch = ((ss-394) % 128)

    if ss >= 650 and ss < 658:

        vel = ((ss-650)+1) * 15

        song_f.append(['note', time, dur, channel, pitch, vel ])

  detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                            output_signature = 'Lars Ulrich Transformer',
                                                            output_file_name = '/content/Lars-Ulrich-Transformer-Seed-Composition',
                                                            track_name='Project Los Angeles',
                                                            list_of_MIDI_patches=[0, 24, 32, 40, 42, 19, 56, 65, 73, 0, 0, 0, 0, 0, 0, 0],
                                                            )

  #=======================================================

  print('Displaying resulting composition...')
  print('=' * 70)

  fname = '/content/Lars-Ulrich-Transformer-Seed-Composition'

  x = []
  y =[]
  c = []

  colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

  block_lines = [(song_f[-1][1] / 1000)]
  block_tokens = [min(len(melody_chords_f), number_of_prime_tokens)]

  for s in song_f:
    x.append(s[1] / 1000)
    y.append(s[4])
    c.append(colors[s[3]])

  if render_MIDI_to_audio:
    FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
    display(Audio(str(fname + '.wav'), rate=16000))

  plt.figure(figsize=(14,5))
  ax=plt.axes(title=fname)
  ax.set_facecolor('black')

  plt.scatter(x,y, c=c)
  plt.xlabel("Time")
  plt.ylabel("Pitch")
  plt.show()

else:
  print('=' * 70)

# (COMPOSITION LOOP)

## Run the cells below in a loop to generate endless continuation

In [ ]:
#@title Standard Continuation Generator

#@markdown Generation settings

number_of_tokens_to_generate = 300 # @param {type:"slider", min:30, max:1025, step:5}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
preview_length_in_tokens = 120 # @param {type:"slider", min:30, max:250, step:5}
number_of_memory_tokens = 2045 #@param {type:"slider", min:400, max:2045, step:5}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}

#@markdown Other settings

render_MIDI_to_audio = True #@param {type:"boolean"}

print('=' * 70)
print('Lars Ulrich Standard Continuation Model Generator')
print('=' * 70)

preview = melody_chords_f[-preview_length_in_tokens:]

mel_cho = melody_chords_f[-number_of_memory_tokens:]

inp = [mel_cho] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.module.generate(inp,
                              number_of_tokens_to_generate,
                              temperature=temperature,
                              return_prime=False,
                              eos_token=None,
                              verbose=True)

out0 = out.tolist()

print('=' * 70)
print('Done!')
print('=' * 70)

#======================================================================

print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:10])
  print('=' * 70)

  if len(out) != 0:

      song = preview + out1
      song_f = []

      time = 0
      dur = 0
      channel = 0
      pitch = 0
      vel = 90

      for ss in song:

        if ss >= 0 and ss < 10:

            channel = ss

        if ss >= 10 and ss < 266:

            time += (ss-10) * 8

        if ss >= 266 and ss < 394:

            dur = (ss-266) * 32

        if ss >= 394 and ss < 650:

            pitch = ((ss-394) % 128)

        if ss >= 650 and ss < 658:

            vel = ((ss-650)+1) * 15

            song_f.append(['note', time, dur, channel, pitch, vel ])

      detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                                output_signature = 'Lars Ulrich Transformer',
                                                                output_file_name = '/content/Lars-Ulrich-Transformer-Composition_'+str(i),
                                                                track_name='Project Los Angeles',
                                                                list_of_MIDI_patches=[0, 24, 32, 40, 42, 19, 56, 65, 73, 0, 0, 0, 0, 0, 0, 0]
                                                                )
      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Lars-Ulrich-Transformer-Composition_'+str(i)

      x = []
      y =[]
      c = []

      colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

      for s in song_f:
        x.append(s[1] / 1000)
        y.append(s[4])
        c.append(colors[s[3]])

      if render_MIDI_to_audio:
        FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
        display(Audio(str(fname + '.wav'), rate=16000))

      plt.figure(figsize=(14,5))
      ax=plt.axes(title=fname)
      ax.set_facecolor('black')

      plt.scatter(x,y, c=c)

      pbl = song_f[(int(preview_length_in_tokens / 5))][1] / 1000

      ax.axvline(x=pbl, c='w')

      plt.xlabel("Time")
      plt.ylabel("Pitch")
      plt.show()

In [ ]:
#@title Choose one generated block to add to the composition
block_action = "add_last_generated_block" #@param ["add_last_generated_block", "remove_last_added_block"]
add_block_with_batch_number = 0 #@param {type:"slider", min:0, max:15, step:1}
render_MIDI_to_audio = False #@param {type:"boolean"}

print('=' * 70)

if block_action == 'add_last_generated_block':
  melody_chords_f.extend(out0[min(len(out0)-1, add_block_with_batch_number)])
  print('Block added!')
else:
  if len(block_tokens) > 1:
    melody_chords_f = melody_chords_f[:(len(melody_chords_f)-block_tokens[-1])]
    print('Block removed!')
  else:
    print('Nothing to remove!!!')

print('=' * 70)
print('Composition now has', (len(melody_chords_f) // 5), 'notes')
print('Composition now has', len(melody_chords_f), 'tokens')


print('=' * 70)
print('Sample INTs', out1[:10])
print('=' * 70)

if len(melody_chords_f) != 0:

    song = melody_chords_f
    song_f = []

    time = 0
    dur = 0
    channel = 0
    pitch = 0
    vel = 90

    for ss in song:

      if ss >= 0 and ss < 10:

          channel = ss

      if ss >= 10 and ss < 266:

          time += (ss-10) * 8

      if ss >= 266 and ss < 394:

          dur = (ss-266) * 32

      if ss >= 394 and ss < 650:

          pitch = ((ss-394) % 128)

      if ss >= 650 and ss < 658:

          vel = ((ss-650)+1) * 15

          song_f.append(['note', time, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                              output_signature = 'Lars Ulrich Transformer',
                                                              output_file_name = '/content/Lars-Ulrich-Transformer-Composition',
                                                              track_name='Project Los Angeles',
                                                              list_of_MIDI_patches=[0, 24, 32, 40, 42, 19, 56, 65, 73, 0, 0, 0, 0, 0, 0, 0]
                                                              )
    print('=' * 70)
    print('Displaying resulting composition...')
    print('=' * 70)

    fname = '/content/Lars-Ulrich-Transformer-Composition'

    x = []
    y =[]
    c = []

    colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

    if block_action == 'add_last_generated_block':
      block_lines.append((song_f[-1][1] / 1000))
      block_tokens.append(len(out0[min(len(out0)-1, add_block_with_batch_number)]))

    else:
      if len(block_tokens) > 1:
        block_lines.pop()
        block_tokens.pop()

    for s in song_f:
      x.append(s[1] / 1000)
      y.append(s[4])
      c.append(colors[s[3]])

    if render_MIDI_to_audio:
      FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
      display(Audio(str(fname + '.wav'), rate=16000))

    plt.figure(figsize=(14,5))
    ax=plt.axes(title=fname)
    ax.set_facecolor('black')

    plt.scatter(x,y, c=c)

    for bl in block_lines:
      ax.axvline(x=bl, c='w')

    plt.xlabel("Time")
    plt.ylabel("Pitch")
    plt.show()

# Congrats! You did it! :)